# Title: Data Engineering Project

Ojbectives & Requirements:

Extract the name of each team

Present the total number of goals scored by each german team during the 2011 season

Present the total number of wins the team earned during the 2011 season

Display histogram visualizations of the teams' wins and losses for the 2011 season (store the visualization directly)

Highlight the teams' win percentage on days where it was raining during games in the 2011 season.

# I. Package Installation

In [10]:
#This is where we import the relevant packages to be used. 
#SQL to organise our data / Pandas & Numpy for the second stage of anlaysis.

import sqlite3 
import pandas as pd
import numpy as np
import requests
import json

# II. Data Join - SQL

In [11]:
#Unique DataFrame created using three data sources: Teams_in_Matches / Unique_Teams / Matches 
#Presents duplicated entries for each match

conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

cur.execute("""SELECT * FROM Teams_in_Matches JOIN Unique_Teams USING (Unique_Team_ID) JOIN Matches 
               USING (Match_ID) WHERE Season = 2011""")
pdset = pd.DataFrame(cur.fetchall())
pdset.columns = [x[0] for x in cur.description]
pdset.head()

,Match_ID,Unique_Team_ID,TeamName,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,1,Bayern Munich,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1092,22,Nurnberg,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,1093,1,Bayern Munich,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,1093,17,Stuttgart,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,1094,1,Bayern Munich,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A


# III. Adding result data and goals of the TeamName - Pandas

In [15]:
#Replace 'TeamName' column Label with 'MainTeam' for clarity

pdset.rename(columns={'TeamName':'MainTeam'}, inplace = True)

#Adding result of the TeamName (we'll refer to TeamName as Main Team)

for i, row in pdset.iterrows():
    if row.MainTeam == row.AwayTeam:
        # Away
        if row.FTR == 'A':
            pdset.loc[i,'Result'] = "W"
        elif row.FTR == 'D':
            pdset.loc[i,'Result'] = "D"
        else:
            pdset.loc[i,'Result'] = "L"
    else:
        # Home
        if row.FTR == 'H':
            pdset.loc[i,'Result'] = "W"
        elif row.FTR == 'D':
            pdset.loc[i,'Result'] = "D"
        else:
            pdset.loc[i,'Result'] = "L"

#Adding number of goals for the main team 
            
pdset['goals_scored'] = np.where(pdset['MainTeam'] == pdset ['AwayTeam'], pdset['FTAG'], pdset['FTHG'])

#Keeping only German football divisions: D1 & D2

pdset = pdset[pdset.Div != 'E0']